In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn.functional as F
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import torch.nn as nn
from torchsummary import summary

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
trans = transforms.Compose([transforms.ToTensor(), 
                            transforms.Resize((224,224)),
                            transforms.Normalize((0.1307,), (0.3081,))])

In [ ]:
class Fasion_Mnist(Dataset):
    def __init__(self, csv_path, transform=None):
        data = pd.read_csv(csv_path)       
        label = data.iloc[:, 0].values
        train_set = data.iloc[:, 1:].values.astype(np.uint8).reshape(len(label), 28, 28)
       
        self.train_set = train_set
        self.train_label = label
        self.transform = transform
    
    def __getitem__(self, index):
        img = self.train_set[index]
        target = int(self.train_label[index])
        if self.transform is not None:
            img = self.transform(img)
        else:
            image = torch.tensor(image/255., dtype=torch.float)
#         label = torch.tensor(label, dtype=torch.long)    
        return img, target
    
    def __len__(self):
        return len(self.train_set)

In [ ]:
train_dataset = Fasion_Mnist(r"C:\Users\14282\OneDrive\桌面\fasion_mnist\fashion-mnist_test.csv", transform=trans)
test_dataset = Fasion_Mnist(r"C:\Users\14282\OneDrive\桌面\fasion_mnist\fashion-mnist_train.csv", transform=trans)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
learning_rate = 0.01
momentum = 0.5
EPOCH = 20

In [ ]:
def vgg_block(in_channels, out_channels, num_convs, kernel_size=3, stride=1, padding=1):
    block = nn.Sequential()
    for i in range(num_convs):
        conv2d = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(True)
            )
        block.add_module(f'conv2d_{i}',conv2d)
        in_channels = out_channels
    block.add_module(f'maxpool_{i}',nn.MaxPool2d(2,2))
    return block

#VGG-16模型搭建：包含5个vgg_block块，3个全连接层
class VGG16(nn.Module):
    def __init__(self, in_dim, n_class) -> None:
        super(VGG16, self).__init__()
        self.features = nn.Sequential(
            
            vgg_block(in_channels=in_dim, out_channels=64, num_convs=2),

            vgg_block(in_channels=64, out_channels=128, num_convs=2),
            
            vgg_block(in_channels=128, out_channels=256, num_convs=3),
            
            vgg_block(in_channels=256, out_channels=512, num_convs=3),

            vgg_block(in_channels=512, out_channels=512, num_convs=3)
        )

        self.classifier = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(True),
            nn.Dropout(),

            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),   
            nn.Linear(4096, n_class)
        )

    def forward(self,x):
        x = self.features(x)
        x = x.view(x.size(0),-1)
        x = self.classifier(x)
        return x

model = VGG16(1, 10).to(device)
summary(model, input_size=(1, 124, 124))

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)

In [ ]:
def train(epoch):
    running_loss = 0.0  
    running_total = 0
    running_correct = 0
    for batch_idx, data in enumerate(train_loader, 0):
        inputs, target = data
        # inputs, target = inputs.cuda(), target.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, dim=1)
        running_total += inputs.shape[0]
        running_correct += (predicted == target).sum().item()        
        
        if batch_idx % 100 == 99:
            print(f'[{epoch + 1}, {batch_idx + 1:5d}]: loss: {running_loss / 300:.3f}, acc: {100 * running_correct / running_total:.2f}%')
            running_loss = 0.0  
            running_total = 0
            running_correct = 0  

In [ ]:
def test():
    correct = 0
    total = 0
    with torch.no_grad():  # 测试集不用算梯度
        for data in test_loader:
            images, labels = data
            # images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, dim=1)  # dim = 1 列是第0个维度，行是第1个维度，沿着行(第1个维度)去找1.最大值和2.最大值的下标
            total += labels.size(0)  # 张量之间的比较运算
            correct += (predicted == labels).sum().item()
    acc = correct / total
    
    return acc

In [ ]:
if __name__ == '__main__':
    acc_list_test = []
    for epoch in range(EPOCH):
        train(epoch)
        if epoch % 10 == 9:  #每训练10轮 测试1次
            acc_test = test()
            acc_list_test.append(acc_test)

    plt.plot(acc_list_test)
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy On TestSet')
    plt.show()